# Interacting with popular databases using Python

The primary mode of interaction with webpage is through the web communication protocol `HTTP` or `HTTPS` for encrypted. Within `HTTP`,
there are different methods for requesting and submitting information. The methods that we will be using today are `GET` and `POST` requests.

## HTTP Resquests

![Graphic example of an HTTP request](https://mdn.mozillademos.org/files/13821/HTTP_Request_Headers2.png)

* `POST` request sends headers information as well as additional form data content.
* `GET` request only sends headers information.

## HTTP Response

![Graphic example of an HTTP response](https://mdn.mozillademos.org/files/13823/HTTP_Response_Headers2.png)


In [1]:
import requests
import json
import pandas as pd
from io import StringIO

`requests` is the recommended module for requesting and sending resources to a web-based API endpoint

`json` is the built-in module for working with text data in JSON format

# UniProt REST API

In [2]:
uniprot_url = "https://www.uniprot.org/uploadlists"
headers = {
    "User-Agent": "Python, toan.phung@uq.net.au"
}
acc_file = "../data/testlist.txt"

`https://www.uniprot.org/uploadlists` is the url of the uniprot REST API that we will used to request information

`headers` is the metadata that should be included with every api requests for potential debugging purpose from uniprot admin

In [3]:
with open(acc_file, "rt") as source_acc:
    l = [i.strip() for i in source_acc]

Opening the file containing our list of Uniprot accession id and store as an string array


In [4]:
parameters = {
    "query": " ".join(l),
    "format": "tab",
    "from": "ACC,ID",
    "to": "REFSEQ_NT_ID"
}


From the `uploadlists` api endpoint, there are a few options we can choose.

For example, one can use Uniprot ability to convert input id format to id format in different databases.
Above, we are created a parameters dictionary to convert from Uniprot accession. The dictionary contain 4 keys.
- `query` value is a string constructed from the array above with each item joined by a space
- `format` the desired return file format
- `from` input format id type (uniprot acc and id)
- `to` output format id type (RefSeq nucleotide sequence id)


In [5]:
response = requests.get(uniprot_url, params=parameters, headers=headers)
print(response.status_code)

200


`response` is the variable containing the request result from Uniprot.

In [38]:
result_refseq_nt = pd.read_csv(StringIO(response.text), sep="\t")
result_refseq_nt.head()

,From,To
0,P25045,NM_001182805.1
1,Q07844,NM_001181854.1
2,P22147,NM_001181038.1
3,P39931,NM_001182137.1
4,P27692,NM_001182366.1


For saving the output from the operation, you can save directly from the response result or from the data frame.


In [ ]:
# Directly from response
with open("../data/result_map.txt", "wb") as map_file:
    map_file.write(response.content)


In [ ]:
# From pandas dataframe
result_refseq_nt.to_csv("../data/result_map.txt", sep="\t")


`result` store uniprot tabulated data in a `pandas` dataframe.

First columne is the original input id and the second column is the corresponding id in the RefSeq nucleotide database.

In [39]:
parameters["to"] = "P_REFSEQ_AC"
response = requests.get(uniprot_url, params=parameters, headers=headers)
result_refseq_ac = pd.read_csv(StringIO(response.text), sep="\t")
result_refseq_ac.head()

,From,To
0,P25045,NP_014025.1
1,Q07844,NP_013066.1
2,P22147,NP_011342.1
3,P39931,NP_013351.1
4,P27692,NP_013703.1


Above, we changed our query parameter to targeting RefSeq protein accession id instead.

---

Now what if we want to get more information from the Uniprot database instead of just doing id coversion.

In [53]:
extra_parameters = ["id", "entry name", "reviewed", "protein names", "organism", "sequence"]
parameters["to"] = "ACC"
parameters["columns"] = ",".join(extra_parameters)
print(parameters["columns"])

id,entry name,reviewed,protein names,organism,sequence


Adding a fifth key to our parameters named
- `columns` string composed of the columns name of desired data corresponding to the ids collection.
Each column name separated by ",". Above we are getting id, entry name, reviewed status, protein names, organism and 
protein sequence from Uniprot.

For all column name accessible through this mode, you can visit https://www.uniprot.org/help/uniprotkb_column_names


In [45]:
response = requests.get(uniprot_url, params=parameters, headers=headers)
result_uniprot = pd.read_csv(StringIO(response.text), sep="\t")
print(result_uniprot.columns)
result_uniprot.head()


,Entry,Entry name,Status,Protein names,Organism,Sequence,yourlist:M201907126746803381A1F0E0DB47453E0216320D51E8FES
0,P25045,LCB1_YEAST,reviewed,Serine palmitoyltransferase 1 (SPT 1) (SPT1) (...,Saccharomyces cerevisiae (strain ATCC 204508 /...,MAHIPEVLPKSIPIPAFIVTTSSYLWYYFNLVLTQIPGGQFIVSYI...,P25045
1,Q07844,RIX7_YEAST,reviewed,Ribosome biogenesis ATPase RIX7,Saccharomyces cerevisiae (strain ATCC 204508 /...,MVKVKSKKNSLTSSLDNKIVDLIYRLLEEKTLDRKRSLRQESQGEE...,Q07844
2,P22147,XRN1_YEAST,reviewed,5'-3' exoribonuclease 1 (EC 3.1.13.-) (DNA str...,Saccharomyces cerevisiae (strain ATCC 204508 /...,MGIPKFFRYISERWPMILQLIEGTQIPEFDNLYLDMNSILHNCTHG...,P22147
3,P39931,SS120_YEAST,reviewed,Protein SSP120,Saccharomyces cerevisiae (strain ATCC 204508 /...,MRFLRGFVFSLAFTLYKVTATAEIGSEINVENEAPPDGLSWEEWHM...,P39931
4,P27692,SPT5_YEAST,reviewed,Transcription elongation factor SPT5 (Chromati...,Saccharomyces cerevisiae (strain ATCC 204508 /...,MSDNSDTNVSMQDHDQQFADPVVVPQSTDTKDENTSDKDTVDSGNV...,P27692


An example for a more extensive parameters is below


In [82]:
parameters = {
    "query": " ".join(l),
    "format": "tab",
    "from": "ACC,ID",
    "to": "ACC",
    "columns": "id,entry name,reviewed,protein names,genes,organism,length," \
                                   "organism-id,go-id,go(cellular component),comment(SUBCELLULAR LOCATION)," \
                                   "feature(TOPOLOGICAL_DOMAIN),feature(GLYCOSYLATION),comment(MASS SPECTROMETRY)," \
                                   "sequence,feature(ALTERNATIVE SEQUENCE),comment(ALTERNATIVE PRODUCTS) ",
}

response = requests.get(uniprot_url, params=parameters, headers=headers)

result_uniprot_extensive = pd.read_csv(StringIO(response.text), sep="\t")
print(result_uniprot_extensive.columns)
result_uniprot_extensive.head()

Index(['Entry', 'Entry name', 'Status', 'Protein names', 'Gene names',
       'Organism', 'Length', 'Organism ID', 'Gene ontology IDs',
       'Gene ontology (cellular component)', 'Subcellular location [CC]',
       'Topological domain', 'Glycosylation', 'Mass spectrometry', 'Sequence',
       'Alternative sequence', 'Alternative products (isoforms)',
       'yourlist:M201907156746803381A1F0E0DB47453E0216320D52DB7D6'],
      dtype='object')


,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Organism ID,Gene ontology IDs,Gene ontology (cellular component),Subcellular location [CC],Topological domain,Glycosylation,Mass spectrometry,Sequence,Alternative sequence,Alternative products (isoforms),yourlist:M201907156746803381A1F0E0DB47453E0216320D52DB7D6
0,P25045,LCB1_YEAST,reviewed,Serine palmitoyltransferase 1 (SPT 1) (SPT1) (...,LCB1 END8 TSC2 YMR296C,Saccharomyces cerevisiae (strain ATCC 204508 /...,558,559292,GO:0004758; GO:0005783; GO:0016021; GO:0017059...,endoplasmic reticulum [GO:0005783]; integral c...,SUBCELLULAR LOCATION: Cytoplasm. Endoplasmic r...,TOPO_DOM 1 49 Lumenal. {ECO:0000269|PubMed:154...,NaN,NaN,MAHIPEVLPKSIPIPAFIVTTSSYLWYYFNLVLTQIPGGQFIVSYI...,NaN,NaN,P25045
1,Q07844,RIX7_YEAST,reviewed,Ribosome biogenesis ATPase RIX7,RIX7 YLL034C,Saccharomyces cerevisiae (strain ATCC 204508 /...,837,559292,GO:0000055; GO:0005524; GO:0005634; GO:0005730...,nucleolus [GO:0005730]; nucleus [GO:0005634]; ...,"SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:...",NaN,NaN,NaN,MVKVKSKKNSLTSSLDNKIVDLIYRLLEEKTLDRKRSLRQESQGEE...,NaN,NaN,Q07844
2,P22147,XRN1_YEAST,reviewed,5'-3' exoribonuclease 1 (EC 3.1.13.-) (DNA str...,XRN1 DST2 KEM1 RAR5 SEP1 SKI1 YGL173C G1645,Saccharomyces cerevisiae (strain ATCC 204508 /...,1528,559292,GO:0000184; GO:0000741; GO:0000932; GO:0000956...,cytoplasm [GO:0005737]; cytoplasmic stress gra...,"SUBCELLULAR LOCATION: Cytoplasm. Cytoplasm, pe...",NaN,NaN,NaN,MGIPKFFRYISERWPMILQLIEGTQIPEFDNLYLDMNSILHNCTHG...,NaN,NaN,P22147
3,P39931,SS120_YEAST,reviewed,Protein SSP120,SSP120 YLR250W L9672.4,Saccharomyces cerevisiae (strain ATCC 204508 /...,234,559292,GO:0000324; GO:0005509; GO:0005737; GO:0005793,cytoplasm [GO:0005737]; endoplasmic reticulum-...,NaN,NaN,NaN,NaN,MRFLRGFVFSLAFTLYKVTATAEIGSEINVENEAPPDGLSWEEWHM...,NaN,NaN,P39931
4,P27692,SPT5_YEAST,reviewed,Transcription elongation factor SPT5 (Chromati...,SPT5 YML010W YM9571.08,Saccharomyces cerevisiae (strain ATCC 204508 /...,1063,559292,GO:0000993; GO:0001042; GO:0001179; GO:0003677...,DSIF complex [GO:0032044]; mitochondrion [GO:0...,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,NaN,NaN,NaN,MSDNSDTNVSMQDHDQQFADPVVVPQSTDTKDENTSDKDTVDSGNV...,NaN,NaN,P27692


Tabulated output from Uniprot does not give isoform sequence. If you want to get their sequence as well, you will have 
to work with fasta output instead of tabulated. An extra parameters is also needed is `include : "yes"`


In [73]:
parameters = {
    "query": " ".join(l),
    "format": "fasta",
    "from": "ACC,ID",
    "to": "ACC",
    "include": "yes",
    "reviewed": "yes"
}
response = requests.get(uniprot_url, params=parameters, headers=headers)

With the fasta file retrieved, we would still need to save it out.


In [74]:
with open("../data/all_isoforms.fasta", "wb") as fasta_file:
    fasta_file.write(response.content)
    

For query not using an id or accession but a more general search, the api endpoint will have to be changed to `https://www.uniprot.org/uniprot`
. Below we are constructing a new query for this endpoint.

In [83]:
uniprot_url = "https://www.uniprot.org/uniprot"
parameters = {
    "query": "glycoprotein",
    "format": "tab",
    "fil": 'organism:"Homo sapiens (Human) [9606]" AND reviewed:yes',
    "columns": "id,entry name,reviewed,protein names,genes,organism,length," \
                                   "organism-id,go-id,go(cellular component),comment(SUBCELLULAR LOCATION)," \
                                   "feature(TOPOLOGICAL_DOMAIN),feature(GLYCOSYLATION),comment(MASS SPECTROMETRY)," \
                                   "sequence,feature(ALTERNATIVE SEQUENCE),comment(ALTERNATIVE PRODUCTS) ",
}


Then execute the query in similar fashion.


In [84]:
response = requests.get(uniprot_url, params=parameters, headers=headers)
result_uniprot_glyco = pd.read_csv(StringIO(response.text), sep="\t")
result_uniprot_glyco.head()

# NCBI API

Similar to UniProt, multiple NCBI APIs can be accessed through manipulation of the URL content.

In [ ]:
result = pd.read_csv(StringIO(response.text), sep="\t")


`result` store uniprot tabulated data in a `pandas` dataframe.


In [112]:
eutil_path = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
query = result_refseq_ac["To"] + "[accn]"
query = "+OR+".join(query)
params = [
    "db=protein",
    "term={}".format(query),
    "usehistory=y"
]
url = "&".join(params)
headers["Content-Type"] = "application/x-www-form-urlencoded"

Now using the query above, we are performing a search in the protein database of uniprot with requests. However because of the URL length
we will have to use HTTP POST request instead of a simple GET request. The different between GET and POST is that beyond the URL and headers,
GET send no additional data. POST requests allows sending of information that are not suitable for URL.

In [113]:
res = requests.post(eutil_path + "esearch.fcgi", data=url, headers=headers)

Similar to a GET request, POST request result can be accessed at `content` and `text` attribute

In [114]:
res.content

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">\n<eSearchResult><Count>926</Count><RetMax>20</RetMax><RetStart>0</RetStart><QueryKey>1</QueryKey><WebEnv>NCID_1_139466391_130.14.18.97_9001_1563328194_1040498119_0MetA0_S_MegaStore</WebEnv><IdList>\n<Id>398365665</Id>\n<Id>398365397</Id>\n<Id>398364771</Id>\n<Id>147921768</Id>\n<Id>42742305</Id>\n<Id>33438880</Id>\n<Id>6325371</Id>\n<Id>6325031</Id>\n<Id>6323953</Id>\n<Id>6322870</Id>\n<Id>6322406</Id>\n<Id>757873419</Id>\n<Id>398366661</Id>\n<Id>398366635</Id>\n<Id>398366631</Id>\n<Id>398366627</Id>\n<Id>398366603</Id>\n<Id>398366593</Id>\n<Id>398366587</Id>\n<Id>398366579</Id>\n</IdList><TranslationSet/><TranslationStack>   <TermSet>    <Term>NP_014025.1[accn]</Term>    <Field>accn</Field>    <Count>1</Count>    <Explode>N</Explode>   </TermSet>   <TermSet>    <Term>NP_013066.1[accn]</Term>    <Field>accn</Fiel

E-search would return an xml document containing the assigned ID for the query to be used for retrieving the result.
The assigned ID is stored in two part, one in the QueryKey tab, another in the WebEnv tab.

If the return result is large, it is suggested to manually grab 500 sequences at a time.

In [90]:
from bs4 import BeautifulSoup

There are many available packages for parsing of xml content. Today, we are going to use `BeautifulSoup`, a package traditionally for parsing webpage.


In [115]:
soup = BeautifulSoup(res.content, features="lxml-xml")
result_count = soup.find("Count").text
print(result_count)


926


First we load the xml data directly into an `BeautifulSoup` object. While HTML and XML is quite similar, there are differences
that require using of the`xml` parsing engine from the package `lxml` instead of the default parser.

With the `history` parameter enable, the `xml` file would contain two important information that we need to acquire, `QueryKey` and `WebEnv`.
`find` function will return the first element with the specified tags.

In [116]:
query_key = soup.find("QueryKey").text
web_env = soup.find("WebEnv").text

Now we can use E-fetch to retrieve the result. E-fetch can return data in different format.
More information on return datatype
https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/

E-fetch can be used to retrieve data directly without going through E-search. However, E-search can be used for more general purpose.

In [117]:
retrieve_params = [
    "db=protein",
    "query_key={}".format(query_key),
    "WebEnv={}".format(web_env),
    "rettype=gb",
    "retmode=xml",
    "retstart=0",
    "retmax=500"
]
retrieve_url = "&".join(retrieve_params)

We are retrieving 500 results at a time instead of everything at once per NCBI recommendation for longer queries. Below we created
the necessary E-Fetch url for retrieving our results.


In [118]:
del headers["Content-Type"]


In [119]:
res = requests.get(eutil_path + "efetch.fcgi?" + retrieve_url, headers=headers)


The output as we have requested is in `xml` format. However, this time, instead of looking for one instance of an elements,
we are looking for all instances of the `GBSeq` element which encapsulated each result. 

In [120]:
def ncbi_xml_parser(res):
    soup = BeautifulSoup(res.content, features="lxml-xml")
    entries = []
    for gb_seq in soup.find_all("GBSeq"):
        entry = {}
        entry["locus"] = gb_seq.find("GBSeq_locus").text
        entry["definition"] = gb_seq.find("GBSeq_definition").text
        entry["id"] = gb_seq.find("GBSeqid").text
        entry["org"] = gb_seq.find("GBSeq_organism").text
        entry["sequence"] = gb_seq.find("GBSeq_sequence").text
        entries.append(entry)
    return entries

entries = ncbi_xml_parser(res)

By storing the result as an array of dictionaries, we can then convert them into a dataframe if you are prefer to work in
the tabulated format. We can also use `Pandas` to convert the result into a dataframe with.

In [121]:
df = pd.DataFrame(entries)
df.head()

,definition,id,locus,org,sequence
0,uncharacterized protein YNR021W [Saccharomyces...,ref|NP_014418.3|,NP_014418,Saccharomyces cerevisiae S288C,msssifgpltgflervnslnapyqalsydeqkamtiwqrvkfynwt...
1,uncharacterized protein YGR054W [Saccharomyces...,ref|NP_011568.3|,NP_011568,Saccharomyces cerevisiae S288C,mssqfflktsqdielfqsyptfeqsntnskdfpvissvlspcgrfl...
2,uncharacterized protein YBR096W [Saccharomyces...,ref|NP_009654.3|,NP_009654,Saccharomyces cerevisiae S288C,mgvctifrwlfaayllssykslpgayfvrfyyyviqnlflpmftgf...
3,uncharacterized protein YPR010C-A [Saccharomyc...,ref|NP_001091638.1|,NP_001091638,Saccharomyces cerevisiae S288C,mrpaqlllntakktsggykipveltplflavgvalcsgtyftykkl...
4,uncharacterized protein YNL208W [Saccharomyces...,ref|NP_014191.2|,NP_014191,Saccharomyces cerevisiae S288C,msanefyssgqqgqynqqnnqertgapnngqygadngnpngerglf...


Now, as an example, we can rewrite what we have written above so that get every single result instead of just the first 500.

In [123]:
result_count = int(result_count)
entries = []
for i in range(0, result_count, 500):
    retrieve_params = [
        "db=protein",
        "query_key={}".format(query_key),
        "WebEnv={}".format(web_env),
        "rettype=gb",
        "retmode=xml",
        "retstart={}".format(i),
        "retmax={}".format(i + 500)
    ]
    retrieve_url = "&".join(retrieve_params)
    res = requests.get(eutil_path + "efetch.fcgi?" + retrieve_url, headers=headers)
    print("Retrieving with entries starting from {} with {}".format(i, res.url))
    entries += ncbi_xml_parser(res)

print(len(entries))
df = pd.DataFrame(entries)


Retrieving with entries starting from 0 with https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&query_key=1&WebEnv=NCID_1_139466391_130.14.18.97_9001_1563328194_1040498119_0MetA0_S_MegaStore&rettype=gb&retmode=xml&retstart=0&retmax=500
Retrieving with entries starting from 500 with https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&query_key=1&WebEnv=NCID_1_139466391_130.14.18.97_9001_1563328194_1040498119_0MetA0_S_MegaStore&rettype=gb&retmode=xml&retstart=500&retmax=1000
926


# The Human Protein Atlas

A more database focus on mapping of the human proteome to current knowledge at various depths.

In [ ]:
atlas_url = "http://www.proteinatlas.org/"

